# Nicholas Shinn Project 1

# Part 1

## Step 1:

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from time import mktime
from datetime import datetime
import re

r = requests.get('https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares')
soup = BeautifulSoup(r.content, 'html5lib')
soup.prettify()

table = soup.find('table')
data = pd.read_html(str(table),flavor='html5lib')

df = {'rank':[], 'x_class': [], 'date': [], 'region': [], 'start_time': [], 'max_time': [], 'end_time': [], 'movie': []}

rows = table.find_all('td')

for i in range(0,len(rows)):
    if (i == 0 or i % 8 == 0):
        df['rank'].append(rows[i].get_text())
    elif (i % 8 == 1):
        df['x_class'].append(rows[i].get_text())
    elif (i % 8 == 2):
        df['date'].append(rows[i].get_text())
    elif (i % 8 == 3):
        df['region'].append(rows[i].get_text())
    elif (i % 8 == 4):
        df['start_time'].append(rows[i].get_text())
    elif (i % 8 == 5):
        df['max_time'].append(rows[i].get_text())
    elif (i % 8 == 6):
        df['end_time'].append(rows[i].get_text())
    else:
        df['movie'].append(rows[i].get_text())
    
data_frame = pd.DataFrame(df, columns = ['rank', 'x_class', 'date', 'region', 'start_time', 'max_time', 'end_time', 'movie'])
data_frame.index = np.arange(1, len(data_frame)+1)
data_frame

This portion of the code first utilizes beautiful soup 4 and pandas to retrieve the html of the website given. Then using the find() function the table with the top 50 solar flares was read into pandas in order to parse the data out of the html text. Because each row has 8 data attributes, I utilized modulo to distribute each line of data into a data frame with preset columns.

## Step 2:

In [2]:
data_frame.drop('movie', 1, inplace = True)
for i, r in data_frame.iterrows():
    data_frame.set_value(i, 'start_datetime', pd.to_datetime(r.loc['date']+' '+r.loc['start_time']))
    data_frame.set_value(i, 'max_datetime', pd.to_datetime(r.loc['date']+' '+r.loc['max_time']))
    data_frame.set_value(i, 'end_datetime', pd.to_datetime(r.loc['date']+' '+r.loc['end_time'])) 
data_frame.drop('date', 1, inplace = True)
data_frame.drop('start_time', 1, inplace = True)
data_frame.drop('max_time', 1, inplace = True)
data_frame.drop('end_time', 1, inplace = True)
columns = list(data_frame)
columns.insert(len(columns)-1, columns.pop(columns.index('region')))
data_frame

## Step 3: 

In [3]:
r = requests.get('https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html')
nasa = BeautifulSoup(r.content ,"html5lib")
nasa.prettify()
table = ((str(nasa).split('\n'))[14:-4])
data_frame_cols = ['start_date', 'start_time', 'end_date', 'end_time', 'start_frequency', 'end_frequency', 'flare_location',
             'flare_region', 'importance', 'cme_date', 'cme_time', 'cpa', 'width', 'speed']
nasa_data = []
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

for i in table:
    row = i.split()
    if '<a' in row[4]:
        row = row[:4] + row[5:]
    if '<a' in row[5]:
        row = row[:5] + row[6:]
    if '<a' in row[9]:
        row = row[:9] + row[10:]
    if '<a' in row[13]:
        row = row[:13] + row[14:]
    if '>' in row[4]:
        left = (row[4].index('>')+1)
        right = (row[4].index('<'))
        row[4] = row[4][left:right]
    if '>' in row[5]:
        left = (row[5].index('>')+1)
        right = (row[5].index('<'))
        row[5] = row[5][left:right]
    if '>' in row[9]:
        left = (row[9].index('>')+1)
        right = (row[9].index('<'))
        row[9] = row[9][left:right]
    if '>' in row[13]:
        left = (row[13].index('>')+1)
        right = (row[13].index('<'))
        row[13] = row[13][left:right]
    row = row[:14]
    nasa_data.extend([row])
    
nasa_data_frame = pd.DataFrame(nasa_data, columns = data_frame_cols)
nasa_data_frame

## Step 4:

In [5]:
nasa_data_frame = nasa_data_frame.replace(to_replace = {'start_frequency': {'????': 'NaN'}, 'end_frequency': {'????': 'NaN'},
                            'flare_location': {'Back|BACK|-+': 'NaN'}, 'flare_region': {'-+': 'NaN'},
                            'flare_classification': {'-+': 'NaN'}, 'cme_date': {'--/--': 'NaN'},
                            'cme_time': {'--:--': 'NaN'}, 'cme_angle': {'-+': 'NaN'},
                            'cme_width': {'-+': 'NaN'}, 'cme_speed': {'-+': 'NaN'}
                            })

nasa_data_frame['is_halo'] = pd.Series('Halo'==nasa_data_frame['cpa'], index = nasa_data_frame.index)
nasa_data_frame.loc[nasa_data_frame.cpa == 'Halo', 'cpa'] = "NA"
nasa_data_frame['lower'] = pd.Series( nasa_data_frame['width'].astype(str).str[0:4].str.contains('&gt;'), index = nasa_data_frame.index)
nasa_data_frame.loc[nasa_data_frame['width'].str.contains('&gt;'), 'width'] = (nasa_data_frame['width'].str)[4:]
nasa_data_frame.loc[nasa_data_frame.cpa == 'Halo', 'cpa'] = "NA"

for i, r in nasa_data_frame.iterrows():
    
    if r.loc['end_date'] != 'NaN':
        nasa_data_frame.set_value(i, 'end_date', r.loc['start_date'][:5] + r.loc['end_date'])
    if r.loc['cme_date'] != 'NaN':
        nasa_data_frame.set_value(i, 'cme_date', r.loc['start_date'][:5] + r.loc['cme_date'])
        
for i, r in nasa_data_frame.iterrows():
    
    if r.loc['start_time'] != 'NaN' and r.loc['start_date'] != 'NaN':
        if r.loc['start_time'] == '24:00':
            nasa_data_frame.set_value(i, 'start_datetime', pd.to_datetime(r.loc['start_date']+' '+'00:00')+ pd.DateOffset(days=1))
        else:
            nasa_data_frame.set_value(i, 'start_datetime', pd.to_datetime(r.loc['start_date']+' '+r.loc['start_time']))
    else:
        nasa_data_frame.set_value(i, 'start_datetime', 'NaN')
        
    if r.loc['end_time'] != 'NaN' and r.loc['end_date'] != 'NaN':
        if r.loc['end_time'] == '24:00':
            nasa_data_frame.set_value(i, 'end_datetime', pd.to_datetime(r.loc['end_date']+' '+'00:00')+ pd.DateOffset(days=1))
        else:
            nasa_data_frame.set_value(i, 'end_datetime', pd.to_datetime(r.loc['end_date']+' '+r.loc['end_time']))
    else:
        dfnasa.set_value(idx, 'end_datetime', 'NaN')
        
    if r.loc['cme_time'] != 'NaN' and r.loc['cme_date'] != 'NaN':
        if r.loc['cme_time'] == '24:00':
            nasa_data_frame.set_value(i, 'cme_datetime', pd.to_datetime(r.loc['cme_date']+' '+'00:00')+ pd.DateOffset(days=1))
        else:
            nasa_data_frame.set_value(i, 'cme_datetime', pd.to_datetime(r.loc['cme_date']+' '+r.loc['cme_time']))
    else:
        nasa_data_frame.set_value(i, 'cme_datetime', 'NaN')
 
nasa_data_frame.drop('start_time', 1, inplace = True)
nasa_data_frame.drop('start_date', 1, inplace = True)
nasa_data_frame.drop('end_time', 1, inplace = True)
nasa_data_frame.drop('end_date', 1, inplace = True)
nasa_data_frame.drop('cme_time', 1, inplace = True)
nasa_data_frame.drop('cme_date', 1, inplace = True)

nasa_data_columns = nasa_data_frame.columns.tolist()
nasa_data_columns = nasa_data_columns[-3:-1]+nasa_data_columns[:-8]+nasa_data_columns[-1:]+nasa_data_columns[-8:-3]
nasa_data_frame = nasa_data_frame[nasa_data_columns]
nasa_data_frame